## Plotly Figure Friday 2024 - Week 33- August 16 - US Presidential Elections
This week's Figure Friday explores the Irish Gender Pay Gap. Work on the following plots:
4x2 heat map showing percentages of males females in each of 4 quantiles
Make 2-point line plots to see if any differences between 2022 and 2023 (only years with data)


In [6]:
# Un-named DEMOCRATIC candidate from ARIZONA in 2016 won 4 votes, drup nulls to remove

import plotly.express as px
import plotly.graph_objects as go
import polars as pl
pl.Config().set_tbl_rows(30)
import numpy as np

df = pl.read_csv(
    './Dataset/1976-2020-president.csv',
    ignore_errors=True
    )

# df_candidates used to make dictionaries where keys are years, values are candidates
# take care of multiple candidates from clinton and bush families
# fix an error with Mitt Romney in 2012. Washington state has his first and last names swapped.
df_candidates = (
    df
    .select(pl.col('candidate', 'year','party_simplified'))
    .filter(pl.col('party_simplified').is_in(['DEMOCRAT', 'REPUBLICAN']))
    .filter(pl.col('candidate') != 'OTHER')
    # Un-named DEMOCRATIC candidate from ARIZONA in 2016 won 4 votes, drup nulls to remove
    .drop_nulls('candidate')
    .with_columns(
        pl.col('candidate')
        .str.replace('BUSH, GEORGE H.W.', 'Bush Senior')
        .str.replace('BUSH, GEORGE W.', 'Bush Junior')
        .str.replace('MITT, ROMNEY', 'ROMNEY, MITT')
        .str.replace('CLINTON, BILL', 'Clinton B.')
        .str.replace('CLINTON, HILLARY', 'Clinton H.'),
        pl.col('year').cast(pl.Int16)
        )
    .with_columns(
        pl.col('candidate')
        .str.split(',')
        .list.slice(0, 1)
        .list.first()
        .str.to_titlecase()
        .str.replace('Mccain', 'McCain')
        # .alias('surname')
        )
    .unique()
    .sort('year')
    .pivot(index='year', on='party_simplified', values='candidate')
)
# print('df_candidates')
# print(df_candidates)

# Group by year and state, then calculate the vote difference between DEMOCRAT and REPUBLICAN
df_votes = (
    df
    .filter(pl.col('party_simplified').is_in(['DEMOCRAT', 'REPUBLICAN']))
    .with_columns(
        pl.col('party_simplified')
        .str.replace('DEMOCRAT', 'DEM_VOTES')
        .str.replace('REPUBLICAN', 'REP_VOTES')
    )
    .rename({'totalvotes':'TOT_VOTES'})
    .pivot(
        index=['year', 'state', 'state_po','TOT_VOTES'],  # , 'candidate'], 
        on ='party_simplified', 
        values='candidatevotes', 
        aggregate_function='sum'
        )
    .with_columns(pl.col('year').cast(pl.Int16))
)
# print('df_votes')
# print(df_votes)

final_df = (
    df_candidates.join(
        df_votes,
        on='year',
        how='left'
    )
    .with_columns(
        DEM_PCT = pl.col('DEM_VOTES') / pl.col('TOT_VOTES'),
        REP_PCT = pl.col('REP_VOTES') / pl.col('TOT_VOTES'),
    )
    .with_columns(
        DIFF_PCT = pl.col('DEM_PCT') -  pl.col('REP_PCT'),
    )
    .rename({'DEMOCRAT': 'DEM', 'REPUBLICAN': 'REP'})
)

# Create the map
customdata=np.stack((final_df['year'], final_df['state'], final_df['DEM'], final_df['REP'],final_df['DIFF_PCT'] ), axis=-1)
for year in sorted(final_df['year'].unique()):
    print(year)
    year_df = final_df.filter(pl.col('year') == year)
    fig = px.choropleth(year_df,
                        geojson="https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json",
                        locationmode='USA-states',
                        locations='state_po',
                        color='DIFF_PCT',
                        scope="usa",
                        color_continuous_scale=px.colors.diverging.RdBu,
                        color_continuous_midpoint=0,
                        range_color=[-0.2,0.2],
                        animation_frame='year',
                        custom_data=['year', 'state', 'DEM', 'REP', 'DIFF_PCT'],
                        title = str(year)
    )
    fig.update_traces(
        hovertemplate =
            '%{customdata[0]}<br>%{customdata[1]}<br>' +
            '%{customdata[2]}(D) vs %{customdata[3]}(R)<br>' +
            'Dem Margin:%{customdata[4]:.1%}%<br>' +
            '<extra></extra>'
    )
    fig.update_layout(
        margin={"r":0, "t":0, "l":0, "b":0},
        )
    fig.show()

1976


1980


1984


1988


1992


...

2004


2008


2012


2016


2020
